In [ ]:
# Imports
import requests
from requests.exceptions import HTTPError
from sys import exit
requests.packages.urllib3.disable_warnings()

In [ ]:
# Constants
APIC = 'https://sandboxapicdc.cisco.com/api'
NAME = 'admin'
PW = 'ciscopsdt',
R_COMMON = {
    'verify': False,
    'timeout': 5
}

In [ ]:
# Request status
def request_status(r):
    print(f'\nURL: {r.url}\n'
          f'Status: {r.status_code} {r.reason}\n')
    try:
        r.raise_for_status()
    except HTTPError as e:
        print(f'{e!r}\n'
              f'{r.text}\n')

In [ ]:
# APIC Login
def apic_login():
    json = {
        'aaaUser': {
            'attributes': {
                'name': 'admin',
                'pwd': 'ciscopsdt'
            }
        }
    }
    url = f'{APIC}/aaaLogin.json'
    r = requests.post(
        url=url,
        json=json,
        **R_COMMON
    )
    
    request_status(r)
    
    token = r.json()['imdata'][0]['aaaLogin']['attributes']['token']
    cookie = {
        'APIC-cookie': token
    }
    
    print(f'Token: {token[:50]}...')
    
    return cookie

In [ ]:
cookie = apic_login()

In [ ]:
# Create a complex object
json = {
    'fvTenant': {
        'attributes': {
            'name': 'a_tenant',
            'descr': 'Tenant #1'
        },
        'children': [
            {
                'fvCtx': {
                    'attributes': {
                        'name': 'vrf_1',
                        'descr': 'VRF #1'
                    }
                }
            },
            {
                'fvBD': {
                    'attributes': {
                        'name': 'bd_1',
                        'descr': 'BD #1'
                    },
                    'children': [
                        {
                            'fvSubnet': {
                                'attributes': {
                                    'ip': '172.16.30.1/24',
                                    'descr': 'Subnet #1'
                                }
                            },
#                            'fvRsCtx': {
#                                'attributes': {
#                                    'tnFvCtxName': 'vrf_1'
#                                }
#                            }
                        }
                    ]
                }
            },
            {
                'fvAp': {
                    'attributes': {
                        'name': 'ap_1',
                        'descr': 'AP #1'
                    },
                    'children': [
                        {
                            'fvAEPg': {
                                'attributes': {
                                    'name': 'epg_1',
                                    'descr': 'EPG #1'
                                },
                                'children': [
                                    {
                                        'fvRsBd': {
                                            'attributes': {
                                                'tnFvBDName': 'bd_1'
                                            }
                                        }
                                    }
                                ]
                            }
                        }
                    ]
                }
            },
            {
                'vzBrCP': {
                    'attributes': {
                        'name': 'contract_1',
                        'descr': 'Contract #1'
                    },
                    'children': [
                        {
                            'vzSubj': {
                                'attributes': {
                                    'name': 'subject_1',
                                    'descr': 'Subject #1'
                                },
                                'children': [
                                    {
                                        'vzRsSubjFiltAtt': {
                                            'attributes': {
                                                'tnVzFilterName': 'filter_1'
                                            }
                                        }
                                    }
                                ]
                            }
                        }
                    ]
                }
            },
            {
                'vzFilter': {
                    'attributes': {
                        'name': 'filter_1',
                        'descr': 'Filter #1'
                    },
                    'children': [
                        {
                            'vzEntry': {
                                'attributes': {
                                    'name': 'entry_1',
                                    'descr': 'Entry #1',
                                    'etherT': 'ip',
                                    'dFromPort': '80',
                                    'dToPort': '80',
                                    'prot': '6'
                                },
                            }
                        }
                    ]
                }
            }
        ]
    }
}

In [ ]:
# Send the request
cookie = apic_login()

url = f'{APIC}/mo/uni.json?rsp-subtree=modified'
r = requests.post(
    url=url,
    cookies=cookie,
    json=json,
    **R_COMMON
)

request_status(r)

In [ ]:
cookie = apic_login()
url = f'{APIC}/mo/uni/tn-a_tenant.json'
r = requests.delete(
    url,
    cookies=cookie,
    **R_COMMON
)

request_status(r)

In [ ]:
r.json()